In [44]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import string
import requests
import collections
import io
import tarfile
import gzip
from nltk.corpus import stopwords
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [45]:
sess = tf.Session()

In [46]:
batch_size = 100       # 한 번에 얼마나 많은 단어를 train 할 지
embedding_size = 100   # 단어당 embedding vector의 길이
vocabulary_size = 2000 # 얼마나 많은 단어를 트레이닝에 쓸 것인지
generations = 100000   # epoch 사이즈
print_loss_every = 1000# 결과값을 1000번에 한 번 보고

num_sampled = int(batch_size/2)
window_size = 5        # 윈도우에 들어갈 단어의 수

In [47]:
# stopword지정
stops = stopwords.words('english')

# test
print_valid_every = 10000
valid_words = ['cliche', 'love','hate', 'silly', 'sad']

In [48]:
save_folder_name = 'temp'
pos_file = os.path.join(save_folder_name, 'rt-polaritydata','rt-polarity.pos')
print(pos_file)

temp\rt-polaritydata\rt-polarity.pos


## 데이터 로딩

In [49]:
def load_movie_data():
    save_folder_name = 'temp'
    pos_file = os.path.join(save_folder_name, 'rt-polaritydata','rt-polarity.pos')
    neg_file = os.path.join(save_folder_name, 'rt-polaritydata','rt-polarity.neg')
    
    # 파일이 이미 있는지 확인하기
    if not os.path.exists(os.path.join(save_folder_name, 'rt-polaritydata')):
        movie_data_url = 'http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz'
        
        # Save tar.gz file
        req = requests.get(movie_data_url, stream=True)
        with open(os.path.join(save_folder_name, 'temp_movie_review_temp.tar.gz'), 'wb') as f:
            for chunk in req.iter_content(chunk_size = 1024):
                if chunk:
                    f.write(chunk)
                    f.flush()
        
        # Extract tar.gz file into temp folder
        tar = tarfile.open(os.path.join(save_folder_name, 'temp_movie_review_temp.tar.gz'), "r:gz")
        tar.extractall(path='temp')
        tar.close()
    
    pos_data = []
    with open(pos_file, 'r', encoding='latin-1') as f:
        for line in f:
            pos_data.append(line.encode('ascii',errors='ignore').decode())
    f.close()
    pos_data = [x.rstrip() for x in pos_data] # rstrip: 오른쪽 공백 지우기
    
    neg_data = []
    with open(neg_file, 'r', encoding='latin-1') as f:
        for line in f:
            neg_data.append(line.encode('ascii',errors='ignore').decode())
    f.close()
    neg_data = [x.rstrip() for x in neg_data]
    
    texts = pos_data + neg_data
    target = [1]*len(pos_data) + [0]*len(neg_data)
    
    return (texts, target)

In [50]:
data = load_movie_data()

In [51]:
texts = data[0]
target = data[1]

## Text에서 필요없는 부분을 떼어내기

In [52]:
def normalize_text(texts, stops):
    # Lower case
    texts = [x.lower() for x in texts]
    
    # Remove punctuation
    texts = [''.join(c for c in x if c not in string.punctuation) for x in texts]
    
    # Remove numbers
    texts = [''.join(c for c in x if c not in '0123456789') for x in texts]
    
    # Remove stopwords
    texts = [' '.join(word for word in x.split() if word not in (stops)) for x in texts]
    
    # Trim extra whitespace
    texts = [' '.join(x.split()) for x in texts]
    
    return texts

In [53]:
texts = normalize_text(texts, stops)

# Texts must contain at least 3 words
target = [target[ix] for ix, x in enumerate(texts) if len(x.split()) >2]
texts = [x for x in texts if len(x.split())>2]

In [54]:
texts[1]

'gorgeously elaborate continuation lord rings trilogy huge column words cannot adequately describe cowriterdirector peter jacksons expanded vision j r r tolkiens middleearth'

## 문장에 있는 단어들을 빈도수 기준 정렬, 등수 번호 부여

In [55]:
# Build dictionary
def build_dictionary(sentences, vocabulary_size):
    # Turn sentences into lists of words
    split_sentences = [s.split() for s in sentences]
    words = [x for sublist in split_sentences for x in sublist]
    
    # Initialize list of [word, word_count] for each word, starting with unknown
    count = [['RARE', -1]]
    
    # 가장 빈도수가 높은 단어들 N개를 이 리스트에 추가
    count.extend(collections.Counter(words).most_common(vocabulary_size-1))
    # counter.most_common(K)을 쓰면 상위 K개의 최빈도 단어들이 아래와 같은 상태로 나옴
    # 예) K=3일 때, [('counter',10),('words',7), ('collections',5)]
    # 단어와 빈도로 이루어진 count 리스트를 딕셔너리 형태로 변환
    word_dict = {}
    for word, word_count in count:
        # 최빈도 단어들에 등수를 부여하는 과정
        # dictionary length는 0, 1, 2, 3.... 이렇게 늘어남
        # 아래의 예시를 보면 알 수 있음
        word_dict[word] = len(word_dict)
    
    return word_dict

#### (참고) Dictionary 예시

In [56]:
a = ['a', 'a', 'c', 'c', 'g', 'g', 'd', 'd', 'd', 'd', 'k', 'k', 'k','k', 'k', 'k']
count1 = [['Rare',-1]]
count1.extend(collections.Counter(a).most_common())
print(count1)

word_dict1 = {}
for word, word_count in count1:
    word_dict1[word] = len(word_dict1)

word_dict1

[['Rare', -1], ('k', 6), ('d', 4), ('a', 2), ('c', 2), ('g', 2)]


{'Rare': 0, 'a': 3, 'c': 4, 'd': 2, 'g': 5, 'k': 1}

## 문장을 단어의 리스트로 바꾸기

In [57]:
def text_to_numbers(sentences, word_dict):
    # initialize the returned data
    data = []
    for sentence in sentences:
        sentence_data = []
        # word_dict에 있는 단어라면 해당 인덱스를, 없는 단어면 0번을 부여
        for word in sentence.split(' '):
            if word in word_dict:
                word_ix = word_dict[word]
            else:
                word_ix = 0
            sentence_data.append(word_ix)
        data.append(sentence_data)
    return data

## 데이터셋을 위의 함수들로 변형하기

In [58]:
word_dictionary = build_dictionary(texts, vocabulary_size)
word_dictionary_rev = dict(zip(word_dictionary.values(), word_dictionary.keys()))
text_data = text_to_numbers(texts, word_dictionary)

# Get validation word keys
valid_examples = [word_dictionary[x] for x in valid_words]

In [59]:
valid_examples

[1490, 28, 940, 205, 359]

## 데이터를 랜덤하게 만들기

In [99]:
def generate_batch_data(sentences, batch_size, window_size, method='skip-gram'):
    # Fill up data batch
    batch_data = []
    label_data = []
    while len(batch_data) < batch_size:
        # select random sentences to start
        rand_sentences = np.random.choice(sentences)
        # Genrate consecutive windows to look at
        window_sequences= [rand_sentences[max((ix-window_size),0):(ix+window_size)] 
                           for ix, x in enumerate(rand_sentences)]
        # Denote which element of each window is the center word of interest
        # 19번째 줄에 대한 설명:
        # window_sequences는 (ix-window_size에서 ix-1까지가 왼쪽 문맥, ix+1부터 ix+window_size까지가 오른쪽 문맥이다)
        # 중심단어는 ix인 셈인데, 문제는 window_size보다 작은 ix들이 padding이 있어야만 중심단어라는데 있다.
        # 하지만 여기서는 과감히 padding을 하지 않고, 일단 중심단어부터 뽑아내고 있다. 
        # window_sequence는 padding이 안 되는 앞 단어들에 대해서는 본인의 index를 쓰도록 하고,
        # 중간의 단어들에 대해서는 length가 target word(1) + window_size *2의 length를 만족시키므로
        # center word의 인덱스로 window_size를 써도 된다. (이 말이 더 어려우므로 밑의 참고를 보자.)
        label_indices = [ix if ix<window_size else window_size for ix,x in enumerate(window_sequences)]
        
        # Pull out center word of interest for each window and create a tuple for each window
        if method=='skip-gram': # 단어로부터 문맥단어들을 유추해내는 방식
            batch_and_labels = [(x[y], x[:y] + x[y+1:]) for x,y in zip(window_sequences, label_indices)]
            # Make it into a big list of tuples (target word, surrounding word)
            tuple_data = [(x, y_) for x,y in batch_and_labels for y_ in y]
        elif method=='cbow':
            batch_and_labels = [(x[:y]+x[y+1:], x[y]) for x,y in zip(window_sequences, label_indices)]
            # Make it into a big list of tuples
            tuple_data = [(x_, y) for x,y in batch_and_labels for x_ in x]
        else:
            raise ValueError('Method {} not implemented yet.'.format(method))
        
        # extract batch and labels
        batch, labels = [list(x) for x in zip(*tuple_data)]
        batch_data.extend(batch[:batch_size])  # uniform한 길이만 쓰기 위해서 batch_size로 끊어낸다
        label_data.extend(labels[:batch_size])
    
    # Trim batch and label at the end
    batch_data = batch_data[:batch_size]
    label_data = label_data[:batch_size]
        
    # Convert to numpy array
    batch_data = np.array(batch_data)
    label_data = np.transpose(np.array([label_data])) # 세로로 세워서 각각의 label data를 리스트로 싼 모양
    
    return (batch_data, label_data)

#### 위의 메커니즘이 이해가 안되면 참고 (특히 label_indices)

In [34]:
sentences = [['i','am','bad'], 
             ['you','are','good'], 
             ['he','is','magnificent','as','fuck'],
             ['are','you','dumb','as','fuck'],
             ['how','on','earth','can','you','do','that']]

window_size = 2
np.random.seed(777)
rand_sentences1 = np.random.choice(sentences)
window_sequences= [rand_sentences1[max((ix-window_size),0):(ix+window_size+1)] 
                           for ix, x in enumerate(rand_sentences1)]
label_indices = [ix if ix<window_size else window_size for ix,x in enumerate(window_sequences)]

In [35]:
window_sequences

[['are', 'you', 'dumb'],
 ['are', 'you', 'dumb', 'as'],
 ['are', 'you', 'dumb', 'as', 'fuck'],
 ['you', 'dumb', 'as', 'fuck'],
 ['dumb', 'as', 'fuck']]

In [38]:
label_indices

[0, 1, 2, 2, 2]

In [41]:
np.transpose(np.array([label_indices]))

array([[0],
       [1],
       [2],
       [2],
       [2]])

이것의 결과를 잘 보자. 
첫 번째와 두 번째 window sequence에서는 length가 1 + window_size*2보다 작다. 
그렇기 때문에 중심 단어는 늘 index그 자체일 수밖에 없다. 
첫 번째의 window_sequence에 대해서 index는 0,
두 번째 window_sequence에 대해서 index는 1이다.
하지만 sequence length가 5인 "are you dumb as fuck"의 경우
window_size를 index로 하면 중심단어인 dumb이 선택된다. 
길이가 sequence length와 같은 것들은 모두 index를 2로 가질 것이라는 것은 예상할 수 있다.

하지만 왜 sequence length가 5보다 작은 뒤의 window sequence들의 경우 그냥 
window_size를 쓸 수 있는 것인가?
그 이유는 indices가 앞에서부터 세어진다는데 있다. 
뒤쪽에 padding이 필요한 window sequence의 경우
뒤쪽이 얼마나 짧아지든 앞쪽에서부터 세어지기 때문에 그냥 window_size를 index로
채택할 수 있는 것이다.

In [39]:
batch_and_labels = [(x[y], x[:y] + x[y+1:]) for x,y in zip(window_sequences, label_indices)]
tuple_data = [(x, y_) for x,y in batch_and_labels for y_ in y]

In [40]:
# skip-gram : 단어에서 문맥추정
tuple_data

[('are', 'you'),
 ('are', 'dumb'),
 ('you', 'are'),
 ('you', 'dumb'),
 ('you', 'as'),
 ('dumb', 'are'),
 ('dumb', 'you'),
 ('dumb', 'as'),
 ('dumb', 'fuck'),
 ('as', 'you'),
 ('as', 'dumb'),
 ('as', 'fuck'),
 ('fuck', 'dumb'),
 ('fuck', 'as')]

## Embedding matrix 만들기 위한 준비과정

In [63]:
# Define embeddings
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0,1.0))

# NCE loss parameters
nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                                             stddev=1.0/np.sqrt(embedding_size)))

nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
# Create data/target placeholders
x_inputs = tf.placeholder(tf.int32, shape=[batch_size])
y_target = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

# Lookup the word embedding
embed = tf.nn.embedding_lookup(embeddings, x_inputs)

In [87]:
# Get loss from prediction
loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                    biases=nce_biases,
                                    labels=y_target,
                                    inputs=embed,
                                    num_sampled=num_sampled,
                                    num_classes=vocabulary_size))

# Create optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

# Cosine similarity between words
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings/norm
# Valid embeddings는 그냥 테스트를 위해 아까 만든 valid samples와 관련된 것이어서 크게 의미두지 말자
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)

# Valid embeddings와 normalized embeddings 사이의 cosine similarity
# 내적을 위해 transpose가 필요함. 
# valid_embeddings :      (5, embedding_size)
# normalized_embeddings : (vocab_size, embedding_size)
# similarity:             (5, vocab_size)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

# Add variable initializer
init = tf.global_variables_initializer()
sess.run(init)

In [91]:
sim_init = sess.run(similarity)
sim_init.shape

(5, 2000)

#### Tf.reduce_sum에서 keep_dims의 의미와 normalizing

In [75]:
a = tf.constant([[[2.0,1.0,2.0],[3.,4.,5.]],[[2.,1.,3.],[5.,6.,7.]],[[0,1.,4.],[2.,1.,4.]]])
sess = tf.InteractiveSession()

In [76]:
a.eval()

array([[[2., 1., 2.],
        [3., 4., 5.]],

       [[2., 1., 3.],
        [5., 6., 7.]],

       [[0., 1., 4.],
        [2., 1., 4.]]], dtype=float32)

In [77]:
a.shape

TensorShape([Dimension(3), Dimension(2), Dimension(3)])

In [78]:
a[0].eval()

array([[2., 1., 2.],
       [3., 4., 5.]], dtype=float32)

In [83]:
norm = tf.sqrt(tf.reduce_sum(tf.square(a[0]), 1,keep_dims=True))
norm.eval()

array([[3.      ],
       [7.071068]], dtype=float32)

In [84]:
normalized = a[0]
normalized.eval()

array([[0.6666667 , 0.33333334, 0.6666667 ],
       [0.42426407, 0.56568545, 0.70710677]], dtype=float32)

위에서 cosine similarity를 구하기 위해 norm을 구할 때, embeddings matrix의 dimension을 생각해볼 필요가 있다. 그 dimension은 [vocab_size, embed_size].
tf.reduce_sum에서 axis를 1로 지정해주었다는 것은 세로줄끼리 연산을 한다는 것이다. 결국 결과값의 dimension은 [vocab_size, 1]이 된다. 

이렇게 구해진 norm으로 나눈다는 것은 위와 같이 계산됨을 의미한다.

## RUNNNNNNN!

In [100]:
# Run the skip-gram model
loss_vec = []
loss_x_vec = []
for i in range(generations):
    batch_inputs, batch_labels = generate_batch_data(text_data, batch_size, window_size)
    feed_dict = {x_inputs: batch_inputs, y_target: batch_labels}
    
    # Run the trainstep
    sess.run(optimizer, feed_dict=feed_dict)
    
    # Return the loss
    if (i+1)% print_loss_every == 0:
        loss_val = sess.run(loss, feed_dict=feed_dict)
        loss_vec.append(loss_val)
        loss_x_vec.append(i+1)
        print("Loss at step {} : {}".format(i+1, loss_val))
        
    # Validation: Print some random words and top 5 related words
    if (i+1)%print_valid_every == 0:
        # 전체 vocab 단어들과 valid examples의 원소와의 similarity를 (5,2000)에 표시한 것이 sim
        sim = sess.run(similarity)    
        for j in range(len(valid_words)):
            valid_word = word_dictionary_rev[valid_examples[j]]
            top_k = 5
            # valid example에 있는 단어와 같은 단어는 제외해야 하므로 0부터 시작 안 함
            nearest = (-sim[j, :]).argsort()[1:top_k+1] 
            log_str = "Nearest to {}:".format(valid_word)
            for k in range(top_k):
                close_word = word_dictionary_rev[nearest[k]]
                score = sim[j, nearest[k]]
                log_str = "%s %s," % (log_str, close_word)
            print(log_str)

Loss at step 1000 : 4.061773777008057
Loss at step 2000 : 4.1783766746521
Loss at step 3000 : 3.935540199279785
Loss at step 4000 : 4.181026458740234
Loss at step 5000 : 4.602260589599609
Loss at step 6000 : 3.6549248695373535
Loss at step 7000 : 3.282228469848633
Loss at step 8000 : 3.7064990997314453
Loss at step 9000 : 3.6334218978881836
Loss at step 10000 : 4.053454875946045
Nearest to cliche: RARE, poignancy, films, proceedings, russian,
Nearest to love: RARE, queen, relationships, history, thriller,
Nearest to hate: emotionally, cloying, tribute, likely, um,
Nearest to silly: revenge, showtime, apparently, impressive, eccentric,
Nearest to sad: slightly, finest, natural, atmosphere, format,
Loss at step 11000 : 3.6322875022888184
Loss at step 12000 : 3.4331555366516113
Loss at step 13000 : 3.484189987182617
Loss at step 14000 : 3.7072174549102783
Loss at step 15000 : 4.718789100646973
Loss at step 16000 : 3.728086471557617
Loss at step 17000 : 3.8656671047210693
Loss at step 1800